In [2]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

In [3]:
tmp <- subset(df, select = -c(deg_cat))           #hot encoding

#define one-hot encoding function
dummy <- dummyVars(" ~ .", data=tmp)

#perform one-hot encoding on data frame
final_df <- data.frame(predict(dummy, newdata=tmp))


final_df$deg_cat <- df$deg_cat


df <- final_df

## Data partition

In [4]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)


TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [5]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    488     488 

## Models

In [6]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [7]:

model_monmlp <- train(x=TrainData, y=TrainClasses, 
               method = "monmlp") 

** Ensemble 1 
Complex eigenvalues found for method = BFGS 
coefficients for function value 0.2618291  :
  [1] -6.829692e+01  2.862276e+02  5.342416e+01 -9.071487e+01  1.842523e+02
  [6] -2.822244e+02 -1.401433e+02 -3.808435e+01  3.799467e+01  8.850622e+01
 [11] -2.539548e+02  5.474814e+00 -3.708155e-01 -2.199092e+00  3.804980e+01
 [16] -3.806065e+01 -3.313892e+01 -6.835035e+01  8.318722e+01  8.613726e+00
 [21] -5.274059e+01  5.270267e+01  1.802853e+02  4.102303e+01 -2.845109e+01
 [26] -2.622014e+01 -1.408152e+02  5.822305e+01 -5.816556e+01  3.922947e+01
 [31] -3.923035e+01 -3.052900e+01  3.049374e+01 -8.328180e-01  4.325693e+00
 [36]  2.207803e+01 -1.253000e+02  1.003693e+02 -1.748404e+00  1.764421e+00
 [41]  2.720760e+02  1.698562e+02  1.890333e+02  1.494174e+02  1.024929e+02
 [46]  8.738491e+01 -3.140359e+01 -1.282636e+02 -1.077671e+02 -4.958022e+01
 [51] -1.579941e+02  8.853031e+00 -2.878731e+02  1.473845e+01  1.111447e+02
 [56]  7.473220e+01 -1.246500e+02 -1.405544e+02 -4.914767e+

In [8]:

model_mlpW <- train(x=TrainData, y=TrainClasses, 
               method = "mlpWeightDecay") 

## Results

In [9]:
print(model_monmlp)
plot(model_monmlp,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Monotone Multi-Layer Perceptron Neural Network 

976 samples
174 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 976, 976, 976, 976, 976, 976, ... 
Resampling results across tuning parameters:

  hidden1  Accuracy   Kappa    
  1        0.7039953  0.4080149
  3        0.7033000  0.4061017
  5        0.6894081  0.3788294

Tuning parameter 'n.ensemble' was held constant at a value of 1
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were hidden1 = 1 and n.ensemble = 1.


In [10]:
print(model_mlpW)
plot(model_mlpW,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Multi-Layer Perceptron 

976 samples
174 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 976, 976, 976, 976, 976, 976, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa    
  1     0e+00  0.7284216  0.4574586
  1     1e-04  0.7219949  0.4458071
  1     1e-01  0.4992805  0.0000000
  3     0e+00  0.7331131  0.4668437
  3     1e-04  0.7320554  0.4645431
  3     1e-01  0.4969071  0.0000000
  5     0e+00  0.7388847  0.4780098
  5     1e-04  0.7401237  0.4805777
  5     1e-01  0.4998108  0.0000000

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 5 and decay = 1e-04.


In [11]:
# collect resamples
results <- resamples(list(monmlp=model_monmlp,  mlpWeightDecay=model_mlpW))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: monmlp, mlpWeightDecay 
Number of resamples: 25 

Accuracy 
                    Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
monmlp         0.6518106 0.6887052 0.7083333 0.7039953 0.7189349 0.7386364    0
mlpWeightDecay 0.7017544 0.7254902 0.7409471 0.7401237 0.7507082 0.7820163    0

Kappa 
                    Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
monmlp         0.3133655 0.3777798 0.4165226 0.4080149 0.4354512 0.4770533    0
mlpWeightDecay 0.4035088 0.4551486 0.4813243 0.4805777 0.5023806 0.5633032    0


## Prediction

In [12]:
pred_mlpW <- predict(model_mlpW,TestData)
pred_mlpW.prob <- predict(model_mlpW,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_mlpW) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast     103      43
   NotFast   86     187
                                         
               Accuracy : 0.6921         
                 95% CI : (0.6455, 0.736)
    No Information Rate : 0.5489         
    P-Value [Acc > NIR] : 1.442e-09      
                                         
                  Kappa : 0.3654         
                                         
 Mcnemar's Test P-Value : 0.0002174      
                                         
            Sensitivity : 0.5450         
            Specificity : 0.8130         
         Pos Pred Value : 0.7055         
         Neg Pred Value : 0.6850         
             Prevalence : 0.4511         
         Detection Rate : 0.2458         
   Detection Prevalence : 0.3484         
      Balanced Accuracy : 0.6790         
                                         
       'Positive' Class : fast           
                                   

Sensitivity          Specificity       Pos Pred Value 
           0.5449735            0.8130435            0.7054795 
      Neg Pred Value            Precision               Recall 
           0.6849817            0.7054795            0.5449735 
                  F1           Prevalence       Detection Rate 
           0.6149254            0.4510740            0.2458234 
Detection Prevalence    Balanced Accuracy 
           0.3484487            0.6790085

In [13]:
roc_obj <- multiclass.roc(TestClasses, as.numeric(pred_mlpW))
roc_obj


Setting direction: controls < cases




Call:
multiclass.roc.default(response = TestClasses, predictor = as.numeric(pred_mlpW))

Data: as.numeric(pred_mlpW) with 2 levels of TestClasses: fast, NotFast.
Multi-class area under the curve: 0.6952

In [14]:
pred_mlpW.prob

,fast,NotFast
30,0.950619698,0.04938026
70,0.004331418,0.99566859
73,0.016997647,0.98300236
75,0.491778642,0.50822145
80,0.280252039,0.71974802
115,0.046432003,0.95356804
122,0.293449134,0.70655084
125,0.279595524,0.72040451
129,0.689179420,0.31082055
132,0.052236654,0.94776338
